In [1]:
import torch
from torch import nn

In [2]:
class DepSepConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        """
        Depthwise Separable Convolution을 구현한 클래스입니다.
        논문에서는 Conv dw / s1, Conv / s1 으로 기재되어 있음

        Args:
            in_channels (int): 입력 채널의 수.
            out_channels (int): 출력 채널의 수.
            stride (int): 컨볼루션 스트라이드 (기본값은 1).
        """
        super().__init__()

        # Depthwise Convolution
        # 강사는 뎁 Dep 이라고 부름
        self.depthwise = nn.Sequential(nn.Conv2d(in_channels,in_channels,3, stride = stride, padding = 1, groups = in_channels, bias=False), # 3x3 역할 분담을 (RGB) 강제로 했더니 학습이 잘 되었다.
                                       nn.BatchNorm2d(in_channels),
                                       nn.ReLU(inplace=True))

        # Pointwise Convolution
        # 강사는 셉 Sep 이라고 부름
        self.pointwise = nn.Sequential(nn.Conv2d(in_channels,out_channels,1, bias=False), # 1x1
                                       nn.BatchNorm2d(out_channels),
                                       nn.ReLU(inplace=True))
    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class MobileNet(nn.Module):
    """
    MobileNet 아키텍처를 구현한 클래스입니다.

    Args:
        alpha (float): 모델의 너비 다운 스케일링 비율 (0.25, 0.5, 0.75, 1.0 등).
        num_classes (int): 분류할 클래스의 수 (기본값은 1000).
    """
    def __init__(self, alpha, num_classes=1000):
        super().__init__()

        # 첫 번째 컨볼루션 레이어
        self.conv1 = nn.Conv2d(3, int(32*alpha), 3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(int(32*alpha))
        self.relu = nn.ReLU(inplace=True)

        # Depthwise Separable Convolution 레이어
        # alpha : width multiplier, narrow하게 만들 수 있도록 만든 옵션. 0.75, 0.5, 0.25로 실험 가능
        self.conv2 = DepSepConv(int(32*alpha), int(64*alpha), stride=1) # 뎁, 셉 초록색 선으로 그은 것. page 131
        self.conv3 = nn.Sequential(DepSepConv(int(64*alpha), int(128*alpha), stride=2), # down sample, 뎁 셉 뎁 셉 두 번, 핑크, 초록 한 쌍
                                   DepSepConv(int(128*alpha), int(128*alpha)))
        self.conv4 = nn.Sequential(DepSepConv(int(128*alpha), int(256*alpha), stride=2), # down sample
                                   DepSepConv(int(256*alpha), int(256*alpha)))
        self.conv5 = nn.Sequential(DepSepConv(int(256*alpha), int(512*alpha), stride=2), # down sample
                                   *[DepSepConv(int(512*alpha), int(512*alpha)) for i in range(5)]) # 다섯번 돌아가는 부분
        self.conv6 = nn.Sequential(DepSepConv(int(512*alpha), int(1024*alpha), stride=2), # down sample
                                   DepSepConv(int(1024*alpha), int(1024*alpha)))

        # Global Average Pooling 및 Fully Connected 레이어
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1)) # GAP, 데이터 사이즈에 무관하게 실험 가능
        self.fc = nn.Linear(int(1024*alpha), num_classes) # Fully connected

        # weights initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [3]:
# 모델 생성
model = MobileNet(alpha=1) # alpha : width multiplier

# 모델 요약 정보 출력
# print(model)
# !pip install torchinfo
from torchinfo import summary
summary(model, input_size=(2,3,224,224), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
MobileNet                                [2, 1000]                 --
├─Conv2d: 1-1                            [2, 32, 112, 112]         864
├─BatchNorm2d: 1-2                       [2, 32, 112, 112]         64
├─ReLU: 1-3                              [2, 32, 112, 112]         --
├─DepSepConv: 1-4                        [2, 64, 112, 112]         --
│    └─Sequential: 2-1                   [2, 32, 112, 112]         --
│    │    └─Conv2d: 3-1                  [2, 32, 112, 112]         288
│    │    └─BatchNorm2d: 3-2             [2, 32, 112, 112]         64
│    │    └─ReLU: 3-3                    [2, 32, 112, 112]         --
│    └─Sequential: 2-2                   [2, 64, 112, 112]         --
│    │    └─Conv2d: 3-4                  [2, 64, 112, 112]         2,048
│    │    └─BatchNorm2d: 3-5             [2, 64, 112, 112]         128
│    │    └─ReLU: 3-6                    [2, 64, 112, 112]         --
├─Sequent

In [4]:
# 랜덤 입력 데이터로 모델 실행 및 출력 크기 확인
x = torch.randn(2,3,224,224)
print(model(x).shape)

torch.Size([2, 1000])


## DepSep conv 와 그냥 conv 파라미터 비교

In [5]:
# 일반적인 Conv2d 레이어의 가중치 수 계산
print(nn.Conv2d(3,16,3).weight.numel())
print(nn.Conv2d(3,3,3,groups=3).weight.numel())
print(nn.Conv2d(3,16,1).weight.numel())

432
27
48
